<a href="https://colab.research.google.com/github/Sra1panasa/NLP/blob/main/Receipt_FormFilling/concepts_receiptff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Concepts_receipt_FormFilling:**

**Tokenizer**
* A tokenizer is in charge of preparing the inputs for a model. The **library contains tokenizers for all the models**